In [1]:
import pandas as pd


### Load the datasets

In [2]:
games = pd.read_csv("data/games.csv")
player_valuations = pd.read_csv("data/player_valuations.csv")
players = pd.read_csv("data/players.csv")
transfers = pd.read_csv("data/transfers.csv")
game_lineups = pd.read_csv("data/game_lineups.csv")
game_events = pd.read_csv("data/game_events.csv")
appearances = pd.read_csv("data/appearances.csv")
club_games = pd.read_csv("data/club_games.csv")
clubs = pd.read_csv("data/clubs.csv")
competitions = pd.read_csv("data/competitions.csv")


### Display basic info

In [3]:
tables = {
    "games": games, "player_valuations": player_valuations, "players": players, "transfers": transfers,
    "game_lineups": game_lineups, "game_events": game_events, "appearances": appearances, "club_games": club_games,
    "clubs": clubs, "competitions": competitions
}


### Checking missing values and duplicates

In [4]:
for name, df in tables.items():
    print(f"\n📌 {name.upper()} TABLE")
    print(df.info())  # Column types and null values
    print(f"Missing values:\n{df.isnull().sum()}")
    print(f"Duplicates: {df.duplicated().sum()}\n")


📌 GAMES TABLE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73151 entries, 0 to 73150
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 73151 non-null  int64  
 1   competition_id          73151 non-null  object 
 2   season                  73151 non-null  int64  
 3   round                   73151 non-null  object 
 4   date                    73151 non-null  object 
 5   home_club_id            73142 non-null  float64
 6   away_club_id            73142 non-null  float64
 7   home_club_goals         73138 non-null  float64
 8   away_club_goals         73138 non-null  float64
 9   home_club_position      50814 non-null  float64
 10  away_club_position      50814 non-null  float64
 11  home_club_manager_name  72321 non-null  object 
 12  away_club_manager_name  72321 non-null  object 
 13  stadium                 72901 non-null  object 
 14  attendance             

### Convert IDs to integer for better performance

In [5]:
id_columns = ["game_id", "player_id", "club_id", "competition_id"]
for col in id_columns:
    for name, df in tables.items():
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype("Int64")

In [6]:
# Identify the primary keys for each table
primary_keys = {
    "games": "game_id",
    "player_valuations": "player_id",
    "players": "player_id",
    "transfers": "player_id",
    "game_lineups": "game_lineups_id",
    "game_events": "game_event_id",
    "appearances": "appearance_id",
    "club_games": "game_id",
    "clubs": "club_id",
    "competitions": "competition_id"
}

# Check for duplicates
duplicate_counts = {}

for table, pk in primary_keys.items():
    duplicate_counts[table] = tables[table][pk].duplicated().sum()

# Display results
for table, count in duplicate_counts.items():
    print(f"Table: {table}, Duplicate {primary_keys[table]} Count: {count}")


Table: games, Duplicate game_id Count: 0
Table: player_valuations, Duplicate player_id Count: 463663
Table: players, Duplicate player_id Count: 0
Table: transfers, Duplicate player_id Count: 69067
Table: game_lineups, Duplicate game_lineups_id Count: 0
Table: game_events, Duplicate game_event_id Count: 0
Table: appearances, Duplicate appearance_id Count: 0
Table: club_games, Duplicate game_id Count: 73151
Table: clubs, Duplicate club_id Count: 0
Table: competitions, Duplicate competition_id Count: 43


## Convert Column Data Types

## Convert the date Column to Datetime

In [7]:
# Convert 'date' column to datetime format
for table_name, df in tables.items():
    for col in df.columns:
        if "date" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

In [8]:
for table_name, df in tables.items():
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)

# Merging Fact Tables for Optimization

## Merged Games & Appearances (fact_match_performance) games + appearances + game_events

In [10]:
fact_match_performance = tables["games"].merge(
    tables["game_events"], on="game_id", how="left"
).merge(
    tables["appearances"], on=["game_id", "player_id"], how="left"
)
fact_match_performance.to_csv("fact_match_performance.csv", index=False)


## Merged Player Transfers & Valuations (fact_player_market)
Combines: player_valuations + transfers

In [12]:
fact_transfers_valuations = tables["transfers"].merge(
    tables["player_valuations"], on=["player_id"], how="left"
)
fact_transfers_valuations.to_csv("fact_transfers_valuations.csv", index=False)

In [13]:
# Processing dim_players
dim_players = tables["players"][[
    "player_id", "first_name", "last_name", "name", "date_of_birth",
    "position", "foot", "height_in_cm", "contract_expiration_date",
    "agent_name", "country_of_birth", "country_of_citizenship"
]].drop_duplicates()
dim_players.to_csv("dim_players.csv", index=False)

# Processing dim_clubs
dim_clubs = tables["clubs"][[
    "club_id", "name", "domestic_competition_id", "total_market_value",
    "squad_size", "average_age", "foreigners_number", "foreigners_percentage",
    "national_team_players", "stadium_name", "stadium_seats", "coach_name"
]].drop_duplicates()
dim_clubs.to_csv("dim_clubs.csv", index=False)

# Processing dim_competitions
dim_competitions = tables["competitions"][[
    "competition_id", "name", "sub_type", "type", "country_id",
    "country_name", "domestic_league_code", "confederation",
    "is_major_national_league"
]].drop_duplicates()
dim_competitions.to_csv("dim_competitions.csv", index=False)

# Processing dim_game_lineups
dim_game_lineups = tables["game_lineups"][[
    "game_lineups_id", "game_id", "player_id", "club_id",
    "player_name", "position", "number", "team_captain"
]].drop_duplicates()
dim_game_lineups.to_csv("dim_game_lineups.csv", index=False)

print("All dimension tables have been processed and saved as CSV.")


All dimension tables have been processed and saved as CSV.
